In [5]:
import os
import time
import glob
from PIL import Image
import io
import numpy as np
import requests
import pandas as pd
from selenium import webdriver
import chromedriver_binary

In [6]:
driver = webdriver.Chrome()
driver.get('https://www.dmm.co.jp/digital/videoa/')
driver.find_element_by_link_text("はい").click()

In [8]:
files = glob.glob('data/*.csv')
for file in files:
    df = pd.read_csv(file)
    # if df['Title'].count()!=50:
    #     continue
    links = df['Link'].tolist()
    dir = 'actress/'+os.path.basename(file).split('.')[0]
    os.makedirs(dir,exist_ok=True)

    for i,link in enumerate(links):
        driver.get(link)
        try:
            elements = driver.find_element_by_id("sample-image-block").find_elements_by_tag_name("a")
        except :
            continue
        for j,element in enumerate(elements):
            name = element.get_attribute('name')
            if not name :
                continue
            try :
                element.click()
            except :
                close = driver.find_element_by_id("close2")
                close.click()
                continue
            time.sleep(1)
            src = driver.find_element_by_id("preview-image").get_attribute('src')
            img = Image.open(io.BytesIO(requests.get(src).content))
            try :
                img.save(dir+f'/sample{i+1}-{j+1}.jpg')
            except OSError :
                pass
            else :
                close = driver.find_element_by_id("close2")
                close.click()

In [9]:
driver.quit()